# Advanced Querying Mongo

Importing libraries and setting up connection

In [3]:
from pymongo import MongoClient
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [4]:
q1 = {"name":"Babelgum"}

list((db["companies"].find(q1, {"name":1, "_id":0})))

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [8]:
q2 = {"number_of_employees":{"$gte":5000}}

list(db["companies"].find(q2, {"name":1, "number_of_employees":1, "_id":0}).limit(20).sort("number_of_employees", -1))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000},
 {'name': 'Sony', 'number_of_employees': 180500},
 {'name': 'LG', 'number_of_employees': 177000},
 {'name': 'Ford', 'number_of_employees': 171000},
 {'name': 'Boeing', 'number_of_employees': 160000},
 {'name': 'Digital Equipment Corporation', 'number_of_employees': 140000},
 {'name': 'Nokia', 'number_of_employees': 125000},
 {'name': 'MItsubishi Electric', 'number_of_employees': 107000}

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [10]:
q3 = {"founded_year": {"$gte": 2000, "$lte": 2005}}

list(db["companies"].find(q3, {"name":1, "founded_year":1, "_id":0}))

[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Omnidrive', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'StumbleUpon', 'founded_year': 2002},
 {'name': 'Gizmoz', 'founded_year': 2003},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Helio', 'founded_year': 2005},
 {'name': 'Plaxo', 'founded_year': 2002},
 {'name': 'Technorati', 'founded_year': 2002},
 {'name': 'AddThis', 'founded_year': 2004},
 {'name': 'Veoh', 'founded_year': 2004},
 {'name': 'Jingle Networks', 'founded_year': 2005},
 {'name': 'Meetup', 'founded_year': 2002},
 {'name': 'Wesabe', 'founded_year': 2005},
 {'name': 'Jangl SMS', 'founded_year': 2005},
 {'name': 'SmugMug', 'founded_year': 2002},
 {'name': 'LifeLock', 'founded_year': 2005},
 {'name': 'Jajah', 'founded_year': 2005},
 {'name': 'YouTube', 'founded_year': 2005},
 {'name': 'Skype', 'founded_year': 2003},
 {'name': 'Pando Networks', 'founded_year': 2004},
 {'name': 'Ikan', 'foun

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [11]:
q4 = {"$and": [{"founded_year": {"$lt": 2010}}, {"ipo.valuation_amount": {"$gt": 100000000}}]}

list(db["companies"].find(q4, {"name":1, "ipo":1, "_id":0}))

[{'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'name': 'Twitter',
  'ipo': {'valuation_amount': 18100000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2013,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NYSE:TWTR'}},
 {'name': 'Yelp',
  'ipo': {'valuation_amount': 1300000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 3,
   'pub_day': 2,
   'stock_symbol': 'NYSE:YELP'}},
 {'name': 'LinkedIn',
  'ipo': {'valuation_amount': 9310000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2011,
   'pub_month': 7,
   'pub_day': 20,
   'stock_symbol': 'NYSE:LNKD'}},
 {'name': 'Brightcove',
  'ipo': {'valuation_amount': 290000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 2,
   'pub_day': 17,
   'stock_symbol': 'NASDAQ:BCOV'}},
 {'name': 'Amazon',
  'ipo': {'valu

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [13]:
q5 = {"$and": [{"founded_year": {"$lt": 2005}}, {"number_of_employees": {"$lt": 1000}}]}

list(db["companies"].find(q5, {"name":1, "number_of_employees":1, "founded_year":1, "_id":0}).sort("number_of_employees", -1).limit(10))

[{'name': 'Infinera Corporation',
  'number_of_employees': 974,
  'founded_year': 2000},
 {'name': 'NorthPoint Communications Group',
  'number_of_employees': 948,
  'founded_year': 1997},
 {'name': '888 Holdings', 'number_of_employees': 931, 'founded_year': 1997},
 {'name': 'Forrester Research',
  'number_of_employees': 903,
  'founded_year': 1983},
 {'name': 'SonicWALL', 'number_of_employees': 900, 'founded_year': 1991},
 {'name': 'Webmetrics', 'number_of_employees': 900, 'founded_year': 1999},
 {'name': 'Cornerstone OnDemand',
  'number_of_employees': 881,
  'founded_year': 1999},
 {'name': 'Yelp', 'number_of_employees': 800, 'founded_year': 2004},
 {'name': 'ZoomInfo', 'number_of_employees': 800, 'founded_year': 2000},
 {'name': 'MySpace', 'number_of_employees': 800, 'founded_year': 2003}]

### 6. All the companies that don't include the `partners` field.

In [16]:
q6 = {"partners": {"$exists": False}}

list(db["companies"].find(q6, {"name":1, "_id":0}))

[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [19]:
q7 = {"category_code": {"$type": "null"}}

list(db["companies"].find(q7, {"name":1, "_id":0}).limit(20))

[{'name': 'Collective'},
 {'name': 'Snimmer'},
 {'name': 'KoolIM'},
 {'name': 'Level9 Media'},
 {'name': 'VidKing'},
 {'name': 'Drigg'},
 {'name': 'SpaceTime'},
 {'name': 'MMDAYS'},
 {'name': 'Touch Clarity'},
 {'name': 'Inside Group'},
 {'name': 'Repeater Store'},
 {'name': 'Tapesh'},
 {'name': 'iPersians'},
 {'name': 'NewPersia'},
 {'name': 'Pyra Labs'},
 {'name': 'Feedmap'},
 {'name': 'NuvoMedia'},
 {'name': 'Intwine'},
 {'name': 'The Weinstein Company'},
 {'name': 'A1Vacations'}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [20]:
q8 = {"$and": [{"number_of_employees": {"$gte":100}}, {"number_of_employees": {"$lte":1000}}]}

list(db["companies"].find(q8, {"name":1, "number_of_employees":1, "_id":0}).limit(20))

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110},
 {'name': 'Livestream', 'number_of_employees': 120},
 {'name': 'Ustream', 'number_of_employees': 250},
 {'name': 'iContact', 'number_of_employees': 300},
 {'name': 'Yelp', 'number_of_employees': 800},
 {'name': 'Dailymotion', 'number_of_employees': 120},
 {'name': 'RockYou', 'number_of_employees': 106},
 {'name': 'Meebo', 'number_of_employees': 200},
 {'name': 'Eventbrite', 'number_of_employees': 200},
 {'name': 'Box', 'number_of_employees': 950},
 {'name': 'Conduit', 'number_of_employees': 215},
 {'name': 'Redfin', 'number_of_employees': 100},
 {'name': 'Simply Hired', 'number_of_employees': 100},
 {'name': 'oDesk', 'number_of_employees': 120},
 {'name': 'PhotoBox', 'number_of_employees': 600},
 {'name': 'Spreadshirt', 'number_of_employees'

### 9. Order all the companies by their IPO price in a descending order.

In [33]:
list(db["companies"].find(projection={"name":1, "ipo.valuation_amount":1, "_id":0}).sort("ipo.valuation_amount", -1).limit(20))

[{'name': 'GREE', 'ipo': {'valuation_amount': 108960000000}},
 {'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000}},
 {'name': 'Groupon', 'ipo': {'valuation_amount': 12800000000}},
 {'name': 'Tencent', 'ipo': {'valuation_amount': 11000000000}},
 {'name': 'Western Digital', 'ipo': {'valuation_amount': 9430000000}},
 {'name': 'LinkedIn', 'ipo': {'valuation_amount': 9310000000}},
 {'name': 'BMC Software', 'ipo': {'valuation_amount': 6000000000}},
 {'name': 'Rackspace', 'ipo': {'valuation_amount': 5440000000}},
 {'name': 'Baidu', 'ipo': {'valuation_amount': 4000000000}},
 {'name': 'TripAdvisor', 'ipo': {'valuation_amount': 3273770000}},
 {'name': 'HomeAway', 'ipo': {'valuation_amount': 3000000000}},
 {'name': 'Zillow', 'ipo': {'valuation_amount': 2550000000}},
 {'name': 'Nielsen', 'ipo': {'valuation_amount': 1600000000}},
 {'name': 'Yelp', 'ipo': {'valu

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [35]:
list(db["companies"].find(projection={"name":1, "number_of_employees":1, "_id":0}).sort("number_of_employees", -1).limit(10))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [47]:
q11 = {"founded_month": {"$gte": 7}}

list(db["companies"].find(q11, {"name": 1, "founded_month":1, "_id":0}).limit(1000))

[{'name': 'Wetpaint', 'founded_month': 10},
 {'name': 'Omnidrive', 'founded_month': 11},
 {'name': 'Digg', 'founded_month': 10},
 {'name': 'Zoho', 'founded_month': 9},
 {'name': 'Joost', 'founded_month': 10},
 {'name': 'Plaxo', 'founded_month': 11},
 {'name': 'Powerset', 'founded_month': 10},
 {'name': 'eBay', 'founded_month': 9},
 {'name': 'Kyte', 'founded_month': 12},
 {'name': 'Thoof', 'founded_month': 12},
 {'name': 'Jingle Networks', 'founded_month': 9},
 {'name': 'Wesabe', 'founded_month': 12},
 {'name': 'SmugMug', 'founded_month': 11},
 {'name': 'LifeLock', 'founded_month': 8},
 {'name': 'Skype', 'founded_month': 8},
 {'name': 'Pando Networks', 'founded_month': 7},
 {'name': 'Ikan', 'founded_month': 12},
 {'name': 'delicious', 'founded_month': 9},
 {'name': 'AllPeers', 'founded_month': 8},
 {'name': 'Wize', 'founded_month': 8},
 {'name': 'SellABand', 'founded_month': 8},
 {'name': 'iContact', 'founded_month': 7},
 {'name': 'MeeVee', 'founded_month': 7},
 {'name': 'blinkx', 'foun

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [ ]:
q12 = {"$and": {"acquisitions": {"gt": 10000000}}}

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [51]:
q13 = {"acquisition.acquired_year": {"$gt": 2010}}

list(db["companies"].find(q13, {"name": 1, "acquisition.acquired_year":1, "_id":0}).sort("acquisition.acquired_year", -1))

[{'name': 'Sense Networks', 'acquisition': {'acquired_year': 2014}},
 {'name': 'Nullsoft', 'acquisition': {'acquired_year': 2014}},
 {'name': 'Alverix', 'acquisition': {'acquired_year': 2014}},
 {'name': 'Wetpaint', 'acquisition': {'acquired_year': 2013}},
 {'name': 'blogTV', 'acquisition': {'acquired_year': 2013}},
 {'name': 'Mashery', 'acquisition': {'acquired_year': 2013}},
 {'name': 'Dailymotion', 'acquisition': {'acquired_year': 2013}},
 {'name': 'adBrite', 'acquisition': {'acquired_year': 2013}},
 {'name': 'Grockit', 'acquisition': {'acquired_year': 2013}},
 {'name': 'Zenfolio', 'acquisition': {'acquired_year': 2013}},
 {'name': 'LongJump', 'acquisition': {'acquired_year': 2013}},
 {'name': 'Docstoc', 'acquisition': {'acquired_year': 2013}},
 {'name': 'LocaModa', 'acquisition': {'acquired_year': 2013}},
 {'name': 'Tutor', 'acquisition': {'acquired_year': 2013}},
 {'name': 'Xobni', 'acquisition': {'acquired_year': 2013}},
 {'name': 'Zimbra', 'acquisition': {'acquired_year': 2013}}

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [54]:
list(db["companies"].find(projection={"name": 1, "founded_year": 1, "_id": 0}).sort("founded_year",-1).limit(20))

[{'name': 'Fixya', 'founded_year': 2013},
 {'name': 'Wamba', 'founded_year': 2013},
 {'name': 'Advaliant', 'founded_year': 2013},
 {'name': 'Fluc', 'founded_year': 2013},
 {'name': 'iBazar', 'founded_year': 2013},
 {'name': 'Gimigo', 'founded_year': 2013},
 {'name': 'SEOGroup', 'founded_year': 2013},
 {'name': 'Clowdy', 'founded_year': 2013},
 {'name': 'WhosCall', 'founded_year': 2013},
 {'name': 'Pikk', 'founded_year': 2013},
 {'name': 'Tongxue', 'founded_year': 2013},
 {'name': 'Shopseen', 'founded_year': 2013},
 {'name': 'VistaGen Therapeutics', 'founded_year': 2013},
 {'name': 'PeekYou', 'founded_year': 2012},
 {'name': 'headr', 'founded_year': 2012},
 {'name': 'Pinger', 'founded_year': 2012},
 {'name': 'Widgetbox', 'founded_year': 2012},
 {'name': 'Mobiluck', 'founded_year': 2012},
 {'name': 'Skydeck', 'founded_year': 2012},
 {'name': 'Simplicant', 'founded_year': 2012}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [55]:
q15 = {"founded_day": {"$lte": 7}}

list(db["companies"].find(q15, {"name": 1, "founded_day": 1, "_id": 0}).sort("acquisition.price_amount",1).limit(10))

[{'name': 'Omnidrive', 'founded_day': 1},
 {'name': 'Geni', 'founded_day': 1},
 {'name': 'Fox Interactive Media', 'founded_day': 1},
 {'name': 'Gizmoz', 'founded_day': 1},
 {'name': 'Facebook', 'founded_day': 1},
 {'name': 'Joost', 'founded_day': 1},
 {'name': 'Babelgum', 'founded_day': 1},
 {'name': 'eBay', 'founded_day': 1},
 {'name': 'Technorati', 'founded_day': 1},
 {'name': 'Mahalo', 'founded_day': 1}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [56]:
q16 = {"$and": [{"category_code":"web"}, {"number_of_employees": {"$gt": 4000}}]}

list(db["companies"].find(q16, {"name": 1, "_id": 0}).sort("number_of_employees",1))

[{'name': 'Expedia'},
 {'name': 'AOL'},
 {'name': 'Webkinz'},
 {'name': 'Rakuten'},
 {'name': 'Los Angeles Times Media Group'},
 {'name': 'Groupon'},
 {'name': 'Yahoo!'},
 {'name': 'eBay'},
 {'name': 'Experian'}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [58]:
q17 = {"$and": [{"acquisition.price_currency_code":"EUR"}, {"acquisition.price_amount": {"$gt": 10000000}}]}

list(db["companies"].find(q17, {"name": 1, "_id": 0}))

[{'name': 'ZYB'},
 {'name': 'Apertio'},
 {'name': 'Greenfield Online'},
 {'name': 'Webedia'},
 {'name': 'Wayfinder'},
 {'name': 'Tuenti Technologies'},
 {'name': 'BioMed Central'}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [60]:
q18 = {"founded_month": {"$lte": 3}}

list(db["companies"].find(q18, {"name": 1, "acquisition":1,"_id": 0}).limit(10))

[{'name': 'StumbleUpon',
  'acquisition': {'price_amount': 29000000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2009/04/13/ebay-unacquires-stumbleupon/',
   'source_description': "StumbleUpon Beats Skype In Escaping EBay's Clutches",
   'acquired_year': 2009,
   'acquired_month': 4,
   'acquired_day': None,
   'acquiring_company': {'name': 'StumbleUpon', 'permalink': 'stumbleupon'}}},
 {'name': 'Gizmoz',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://www.gizmoz.com',
   'source_description': 'TechCrunch',
   'acquired_year': 2009,
   'acquired_month': 12,
   'acquired_day': 15,
   'acquiring_company': {'name': 'Daz 3d', 'permalink': 'daz-3d'}}},
 {'name': 'Facebook', 'acquisition': None},
 {'name': 'Helio',
  'acquisition': {'price_amount': 39000000,
   'price_currency_code': 'USD',
   'term_code': 'stock',
   'source_url': 'http://news.cnet.com/8301-10784_3-9978

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [ ]:
q19 = {"$and": {"acquisition.acquired_year": {"$ne": {"lte": 2011}}}}

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code